In [ ]:
!pip install awscli

In [ ]:
!aws s3 cp s3://intelinair-data-releases/agriculture-vision/cvpr_challenge_2021/supervised supervised --no-sign-request --recursive

download: s3://intelinair-data-releases/agriculture-vision/cvpr_challenge_2021/supervised/Agriculture-Vision-2021.tar.gz to supervised/Agriculture-Vision-2021.tar.gz


In [ ]:
# prompt: go to the supervised folder and unzip Agriculture-Vision-2021.tar.gz

import os

# Navigate to the supervised directory
os.chdir('supervised')

# Unzip the file
!tar -xzf Agriculture-Vision-2021.tar.gz


^C


In [ ]:

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline
import os
import tensorflow as tf

import pandas as pd

In [ ]:
import os
import csv

# Define the path to the main directory
main_directory = "/content/supervised/Agriculture-Vision-2021/train/labels"

# Get the list of folders (labels)
labels = ["cloud_shadow", "double_planter", "planter_skip", "standing_water", "waterway", "weed_cluster"]

# Prepare the output CSV file
output_file = "image_labels.csv"

# Initialize the data to write
data = []

# Loop through each label folder
for label in labels:
    folder_path = os.path.join(main_directory, label)

    # Check if the folder exists
    if os.path.isdir(folder_path):
        # List all files in the folder
        for image_name in os.listdir(folder_path):
            if os.path.isfile(os.path.join(folder_path, image_name)):
                # Create a row for the CSV
                row = [image_name] + [1 if label == lbl else 0 for lbl in labels]
                data.append(row)

# Write the data to the CSV
with open(output_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["index"] + labels)
    # Write the data rows
    writer.writerows(data)

print(f"CSV file '{output_file}' has been created.")


CSV file 'image_labels.csv' has been created.


In [ ]:
#load csv, train, val

base_dir =  '/content/supervised/Agriculture-Vision-2021/'
train_folder = os.path.join(base_dir+ 'Agriculture-Vision/train/images/rgb')

val_folder = os.path.join(base_dir +'Agriculture-Vision/val/rgb/')


test_folder = os.path.join(base_dir + '/Agriculture-Vision/test/test_rgb/')


train = pd.read_csv('/content/supervised/image_labels.csv')

#strip train index
train['index'] = train['index'].str.rstrip('.png')


In [ ]:
train_2k = train.loc[0:1999]


train_2k.shape

(2000, 7)

In [ ]:
train_image = []
for i in tqdm(range(2000)):
    img = image.load_img('/content/supervised/Agriculture-Vision-2021/train/images/rgb/' + train['index'][i] + '.jpg', color_mode='rgb', target_size=None)
    img = image.img_to_array(img)
    train_image.append(img)

100%|██████████| 2000/2000 [00:16<00:00, 120.74it/s]


In [ ]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os

from tensorflow.keras.models import Model



import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import pandas as pd

In [ ]:
BASE_DIR ='/content/supervised/Agriculture-Vision-2021/train/images/'

rgb= os.path.join(BASE_DIR, 'rgb/')

cloud_shadow = os.path.join(BASE_DIR, 'cloud_shadow')

standing_water = os.path.join(BASE_DIR, 'standing_water')

double_plant = os.path.join(BASE_DIR,'double_plant')

planter_skip = os.path.join(BASE_DIR + 'planter_skip')

water_way = os.path.join(BASE_DIR,'waterway')

weed_cluster = os.path.join(BASE_DIR, 'weed_cluster')





#test dir
TEST_DIR = '/content/supervised/Agriculture-Vision-2021/test/images/'
test_set = os.path.join(TEST_DIR, 'smaller_test_set')

VAL_DIR = '/content/supervised/Agriculture-Vision-2021/val/images/rgb/'

In [ ]:
# Training parameters
batch_size = 6  # orig paper trained all networks with batch_size=128
epochs = 20
data_augmentation = True
num_classes = 6


input_shape = (256, 256, 3)

In [ ]:
subtract_pixel_mean = True
n = 3
# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1
# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

In [ ]:
model_type = 'ResNet%dv%d' % (depth, version)
train_df = pd.read_csv(r'/content/supervised/image_labels.csv')
#strip train index
train_df['index'] = train_df['index'].str.rstrip('.png')
train_df['index'] = train_df['index'] + '.jpg'

columns = ['cloud_shadow', 'double_planter', 'planter_skip', 'standing_water', 'waterway', 'weed_cluster']

In [ ]:
train_df

,index,cloud_shadow,double_planter,planter_skip,standing_water,waterway,weed_cluster
0,2GLBAZRT1_7107-5490-7619-6002.jpg,0,0,1,0,0,0
1,UYFIGY9HM_5811-1072-6323-1584.jpg,0,0,1,0,0,0
2,1BFUWHD9T_7375-1518-7887-2030.jpg,0,0,1,0,0,0
3,HMIWP3ZZG_4816-6889-5328-7401.jpg,0,0,1,0,0,0
4,PHKBDKD9R_8088-5047-8600-5559.jpg,0,0,1,0,0,0
...,...,...,...,...,...,...,...
170827,KVYXUMZUJ_6389-5229-6901-5741.jpg,0,0,0,0,0,1
170828,DFAEFMUTR_8364-4623-8876-5135.jpg,0,0,0,0,0,1
170829,NYIXWWPDF_4398-8600-4910-9112.jpg,0,0,0,0,0,1
170830,DGHJCTDEX_9665-3440-10177-3952.jpg,0,0,0,0,0,1


In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.10) # set validation split

train_generator = train_datagen.flow_from_dataframe(
    dataframe = train_df,
    directory = '/content/supervised/Agriculture-Vision-2021/train/images/rgb/',
    x_col = "index",
    y_col = columns,
    batch_size=8,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(256, 256),
    subset='training'
) # set as training data



validation_generator = train_datagen.flow_from_dataframe(
    dataframe= train_df,
    directory = '/content/supervised/Agriculture-Vision-2021/train/images/rgb/',
    x_col = "index",
    y_col = columns,
    batch_size=8,
    seed=42,
    shuffle = True,
    class_mode="raw",
    target_size = (256, 256),
    subset = 'validation')

Found 153749 validated image filenames.
Found 17083 validated image filenames.


In [ ]:
from keras import regularizers, optimizers

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(256,256,3), activation='relu')) # Define activation here
#model.add(Activation('relu')) # Remove this line
model.add(Conv2D(32, (3, 3), activation='relu')) # Define activation here
#model.add(Activation('relu')) # Remove this line
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu')) # Define activation here
#model.add(Activation('relu')) # Remove this line
model.add(Conv2D(64, (3, 3), activation='relu')) # Define activation here
#model.add(Activation('relu')) # Remove this line
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu')) # Define activation here
#model.add(Activation('relu')) # Remove this line
model.add(Dropout(0.5))
model.add(Dense(6, activation='sigmoid'))
# Change 'lr' to 'learning_rate' in the Adam optimizer
model.compile(optimizers.Adam(learning_rate=0.0001, decay=1e-6),loss="categorical_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [ ]:
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)                    │ (None, 256, 256, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 254, 254, 32)        │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 127, 127, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 125, 125, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 246016)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │     125,960,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │           3,078 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 126,029,350 (480.76 MB)

 Trainable params: 126,029,350 (480.76 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# checkpoint
filepath="model.keras"  # Change the extension to .keras
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size

model.fit(
    train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    validation_data=validation_generator,
    epochs=5,
    callbacks=callbacks_list
)

Epoch 1/5
19218/19218 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.3713 - loss: 1.0872
Epoch 1: val_accuracy improved from -inf to 0.00000, saving model to 50_epochs_best_adam.keras
19218/19218 ━━━━━━━━━━━━━━━━━━━━ 1486s 77ms/step - accuracy: 0.3713 - loss: 1.0872 - val_accuracy: 0.0000e+00 - val_loss: 1.3321
Epoch 2/5
    1/19218 ━━━━━━━━━━━━━━━━━━━━ 29:55 93ms/step - accuracy: 0.1250 - loss: 1.0995

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 2: val_accuracy did not improve from 0.00000
19218/19218 ━━━━━━━━━━━━━━━━━━━━ 113s 6ms/step - accuracy: 0.1250 - loss: 1.0995 - val_accuracy: 0.0000e+00 - val_loss: 1.3332
Epoch 3/5
 4621/19218 ━━━━━━━━━━━━━━━━━━━━ 16:59 70ms/step - accuracy: 0.3698 - loss: 1.0863

KeyboardInterrupt: 

In [ ]:
# prompt: save it into my drive


!mkdir -p /content/drive/MyDrive/colab_files
!cp /content/supervised/model.keras /content/drive/MyDrive/colab_files

In [ ]:
#Or Import the model here
#from keras.models import load_model
#model = load_model('')
model.save("model.h5")

In [ ]:
import os
import csv

# Define the path to the main directory
main_directory = "/content/supervised/Agriculture-Vision-2021/val/labels"

# Get the list of folders (labels)
labels = ["cloud_shadow", "double_planter", "planter_skip", "standing_water", "waterway", "weed_cluster"]

# Prepare the output CSV file
output_file = "val_labels.csv"

# Initialize the data to write
data = []

# Loop through each label folder
for label in labels:
    folder_path = os.path.join(main_directory, label)

    # Check if the folder exists
    if os.path.isdir(folder_path):
        # List all files in the folder
        for image_name in os.listdir(folder_path):
            if os.path.isfile(os.path.join(folder_path, image_name)):
                # Create a row for the CSV
                row = [image_name] + [1 if label == lbl else 0 for lbl in labels]
                data.append(row)

# Write the data to the CSV
with open(output_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["index"] + labels)
    # Write the data rows
    writer.writerows(data)

print(f"CSV file '{output_file}' has been created.")

In [ ]:
val_df = pd.read_csv('/content/supervised/val_labels.csv')

val_df.drop(columns='Unnamed: 0', inplace=True)

#strip train index
val_df['index'] = val_df['index'].str.rstrip('.png')

val_df['index'] = val_df['index'] + '.jpg'

columns = ['cloud_shadow', 'double_planter', 'planter_skip', 'standing_water', 'waterway', 'weed_cluster']

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255) # set validation split

val_generator = train_datagen.flow_from_dataframe(
    dataframe = val_df,
    directory = '/content/supervised/Agriculture-Vision-2021/val/images/rgb/',
    x_col = "index",
    batch_size=1,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(256, 256),
)

In [ ]:
preds = model.predict(val_generator)

In [ ]:
y_val_true = val_df.drop(columns='index').values.argmax(axis=1)
y_val_pred = preds.argmax(axis=1)

In [ ]:
# Importing the metrics package from sklearn library
from sklearn import metrics
# Creating the confusion matrix
cm = metrics.confusion_matrix(y_val_true, y_val_pred)

In [ ]:
# Assigning columns names
cm_df = pd.DataFrame(cm,
            columns = ['cloud_shadow', 'double_planter', 'planter_skip', 'standing_water', 'waterway', 'weed_cluster'],
            index = ['cloud_shadow', 'double_planter', 'planter_skip', 'standing_water', 'waterway', 'weed_cluster'])
# Showing the confusion matrix
cm_df

In [ ]:
metrics.accuracy_score(y_val_true, y_val_pred)

In [ ]:
target_names =  ['cloud_shadow', 'double_planter', 'planter_skip', 'standing_water', 'waterway', 'weed_cluster']

from sklearn.metrics import classification_report
print(classification_report(y_val_true, y_val_pred, target_names=target_names))